In [85]:
import numpy as np;
import os;
import util;
import visualize;

def comparativeLossViz(img_dirs,file_post,loss_post,range_batches,range_images,out_file_html,dir_server,img_caption_pre=None):
    img_files_all=[];
    captions_all=[];
    if img_caption_pre is not None:
        assert len(img_caption_pre)==len(img_dirs);
    batch_str=True;
    if range_batches is None:
        range_batches=range(1);
        batch_str=False;
    
    for batch_num in range_batches:
    # range(1,num_batches+1):
        for im_num in range_images:
            files_curr_list=[];
            captions_curr_list=[];
            for idx_img_dir,img_dir in enumerate(img_dirs):
                if loss_post is None:
                    loss_str='';
                else:
                    loss_all=np.load(os.path.join(img_dir,str(batch_num)+loss_post));
                    if im_num>loss_all.shape[0]:
                        continue;
                    loss_curr=loss_all[im_num-1,0];
                    loss_str="{:10.4f}".format(loss_curr);
                if not batch_str:
                    files_curr=[os.path.join(img_dir,str(im_num)+file_post_curr) for file_post_curr in file_post];
                else:
                    files_curr=[os.path.join(img_dir,str(batch_num)+'_'+str(im_num)+file_post_curr) for file_post_curr in file_post];
                files_curr=[util.getRelPath(file_curr,dir_server) for file_curr in files_curr];
                captions_curr=[os.path.split(file_curr)[1]+' '+loss_str for file_curr in files_curr];
                if img_caption_pre is not None:
                    captions_curr=[img_caption_pre[idx_img_dir]+' '+caption_curr for caption_curr in captions_curr];
                files_curr_list.extend(files_curr);
                captions_curr_list.extend(captions_curr);
            img_files_all.append(files_curr_list);
            captions_all.append(captions_curr_list);
    visualize.writeHTML(out_file_html,img_files_all,captions_all,224,224);

In [22]:
dir_server='/home/SSD3/maheen-data';
dir_meta='/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05'
# range_batches=range(1,2);
# batch_size=60;
range_images=range(1,100);
img_dir_meta=os.path.join(dir_meta,'over_time_train');
range_models=range(75,5000,75*8);
img_dir=[os.path.join(img_dir_meta,str(range_model_curr)) for range_model_curr in range_models]

file_post=['_org.jpg','_warp.jpg'];
out_file_html=img_dir_meta+'.html';
print out_file_html
img_caption_pre=[str(model_num) for model_num in range_models];
comparativeLossViz(img_dir,file_post,None,None,range_images,out_file_html,dir_server,img_caption_pre)


/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05/over_time_train.html


In [15]:
dir_server='/home/SSD3/maheen-data';
dir_meta='/home/SSD3/maheen-data/horse_project/tps_train_allKP_adam_noBad'

dir_meta='/home/SSD3/maheen-data/temp/tps_train_allKP_adam_noBad_bn'
dir_loss=os.path.join(dir_meta,'intermediate');
loss_pre='loss_all_';
loss_post_val='_val.npy';
loss_post_normal='.npy';
test_after=30;

loss_files=[file_curr for file_curr in os.listdir(dir_loss) if file_curr.startswith(loss_pre)];
loss_nums=[int(file_curr[:file_curr.rindex('.')].split('_')[2]) for file_curr in loss_files];
print min(loss_nums),max(loss_nums);
loss_num=max(loss_nums);
loss_file=os.path.join(dir_loss,loss_pre+str(loss_num)+loss_post_normal);
loss_file_val=os.path.join(dir_loss,loss_pre+str(loss_num)+loss_post_val);
loss=np.load(loss_file);
loss_val=np.load(loss_file_val);
iter_loss=range(loss.size);
iter_val=range(0,loss_val.size*test_after,test_after);
loss_val_new=[loss_curr for loss_curr in loss_val if loss_curr<1];
iter_val=[loss_curr for idx,loss_curr in enumerate(iter_val) if loss_val[idx]<1];
loss_val=loss_val_new;

out_file_score=os.path.join(dir_meta,'loss.png');
print out_file_score.replace(dir_server,'vision1.idav.ucdavis.edu:1000');
visualize.plotSimple([(iter_loss,loss),(iter_val,loss_val)],\
                     out_file_score,title='Score Loss at '+str(iter_loss[-1]),xlabel='Iterations',\
                     ylabel='Loss',legend_entries=['Train','Val'])




300 2100
vision1.idav.ucdavis.edu:1000/temp/tps_train_allKP_adam_noBad_bn/loss.png


In [20]:
import random;
# horse_path='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_train_allKP.txt';
# human_path='/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP.txt';

horse_path = '/home/SSD3/maheen-data/horse_project/horse_resize/matches_5_train_fiveKP.txt';
human_path = '/home/SSD3/maheen-data/horse_project/aflw/matches_5_train_fiveKP_noIm.txt';

horse_data=util.readLinesFromFile(horse_path);
human_data=util.readLinesFromFile(human_path);
idx_keep=range(0,len(horse_data),5);
print len(horse_data);
print len(idx_keep);
horse_data=[horse_data[idx] for idx in idx_keep];
human_data=[human_data[idx] for idx in idx_keep];
shuffle_idx=range(len(horse_data));
random.shuffle(shuffle_idx);
horse_data=[horse_data[idx] for idx in shuffle_idx];
human_data=[human_data[idx] for idx in shuffle_idx];

for file_curr,data_curr in [(horse_path,horse_data),(human_path,human_data)]:
    out_file_curr=file_curr[:file_curr.rindex('.')]+'_select.txt';
    print out_file_curr;
    util.writeFile(out_file_curr,data_curr);
    







4720
944
/home/SSD3/maheen-data/horse_project/horse_resize/matches_5_train_fiveKP_select.txt
/home/SSD3/maheen-data/horse_project/aflw/matches_5_train_fiveKP_noIm_select.txt


In [21]:
# path_models='/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05/intermediate/'
dir_meta='/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05'
path_to_models=os.path.join(dir_meta,'intermediate')
out_dir_meta=os.path.join(dir_meta,'over_time_train');
util.mkdir(out_dir_meta);

model_pre=os.path.join(path_to_models,'model_all_');
model_post='.dat'
range_models=range(75,5000,75*8);
print range_models

path_to_th='/home/maheenrashid/Downloads/horses/torch/test_xiuye.th';
out_file_sh=out_dir_meta+'.sh'
commands=[];
for model_idx,model_num in enumerate(range_models):
    model_curr=model_pre+str(model_num)+model_post;
    assert os.path.exists(model_curr);
    out_dir_curr=os.path.join(out_dir_meta,str(model_num));
    command=['th',path_to_th];
    command.extend(['-kp_net_file',model_curr]);
    command.extend(['out_dir',out_dir_curr]);
    command=' '.join(command);
    commands.append(command);
# print commands
util.writeFile(out_file_sh,commands)
print out_file_sh
    



[75, 675, 1275, 1875, 2475, 3075, 3675, 4275, 4875]
/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05/over_time_train.sh


In [6]:
dir_server='/home/SSD3/maheen-data';
dir_meta='/home/SSD3/maheen-data/temp/tps_train_allKP_adam_noBad_bn/'
range_batches=range(1,2);
# batch_size=60;
range_images=range(1,94);
img_dir=[os.path.join(dir_meta,'test_viz')];
# range_models=range(75,5000,75*8);
# img_dir=[os.path.join(img_dir_meta,str(range_model_curr)) for range_model_curr in range_models]

file_post=['_horse.jpg','_gtwarp.jpg','_predwarp.jpg'];
out_file_html=img_dir[0]+'.html';
print out_file_html.replace(dir_server,'http://vision1.idav.ucdavis.edu:1000')
# img_caption_pre=[str(model_num) for model_num in range_models];
comparativeLossViz(img_dir,file_post,None,range_batches,range_images,out_file_html,dir_server,None);

# comparativeLossViz(img_dirs,file_post,loss_post,range_batches,range_images,out_file_html,dir_server,img_caption_pre=None)

http://vision1.idav.ucdavis.edu:1000/temp/tps_train_allKP_adam_noBad_bn/test_viz.html


In [32]:
dir_server='/home/SSD3/maheen-data';
dir_meta='/home/SSD3/maheen-data/horse_project/ft_horse_allKp_gtWarp_halfBack_1e-2/debug'
dir_meta='/home/SSD3/maheen-data/horse_project/ft_horse_allKp_gtWarp_halfBack_1e-4'

dir_meta='/home/SSD3/maheen-data/horse_project/ft_horse_allKp_gtWarp_halfBack_1e-3';
# /1064/test_images/log_test.txt  /home/SSD3/maheen-data/horse_project/ft_horse_allKp_gtWarp_halfBack_1e-3/560/test_images/log_test.txt
# /home/SSD3/maheen-data/horse_project/ft_horse_allKp_gtWarp_halfBack_1e-3/1680/test_images/log_test.txt  /home/SSD3/maheen-data/horse_project/ft_horse_allKp_gtWarp_halfBack_1e-3/final/test_images/log_test.txt

range_batches=range(1,2);
# [5,10];
# batch_size=60;
range_images=range(1,101);
# img_dir=[dir_meta];

# img_caption_pre=['final','1680','1064','560'];
img_caption_pre=['1680','ft_orglr_20epoch'];
img_dir=[os.path.join(dir_meta,img_caption_pre_curr,'test_images') for img_caption_pre_curr in img_caption_pre];

# range_models=range(75,5000,75*8);
# img_dir=[os.path.join(img_dir_meta,str(range_model_curr)) for range_model_curr in range_models]

# file_post=['_gt.jpg','_gtwarp.jpg'];
file_post=['_org.jpg']
# ,'.jpg']
# ,'.jpg']
out_file_html=os.path.join(dir_meta,'comparison_withFT')+'.html';
print out_file_html.replace(dir_server,'http://vision1.idav.ucdavis.edu:1000')
# img_caption_pre=[str(model_num) for model_num in range_models];
comparativeLossViz(img_dir,file_post,None,range_batches,range_images,out_file_html,dir_server,img_caption_pre);


http://vision1.idav.ucdavis.edu:1000/horse_project/ft_horse_allKp_gtWarp_halfBack_1e-3/comparison_withFT.html


In [29]:
dir_server='/home/SSD3/maheen-data';
dir_meta='/home/SSD3/maheen-data/temp/distorted_humans'

range_batches=range(1,2);
range_images=range(1,101);

img_caption_pre=['face_org_40','face_vert_40','face_horz_40'];
img_dir=[os.path.join(dir_meta,img_caption_pre_curr,'test_images') for img_caption_pre_curr in img_caption_pre];

file_post=['_pred.jpg']
# ,'.jpg']
# ,'.jpg']
out_file_html=os.path.join(dir_meta,'comparison_40')+'.html';
print out_file_html.replace(dir_server,'http://vision1.idav.ucdavis.edu:1000')
comparativeLossViz(img_dir,file_post,None,range_batches,range_images,out_file_html,dir_server,img_caption_pre);


http://vision1.idav.ucdavis.edu:1000/temp/distorted_humans/comparison_40.html


In [4]:
dir_server='/home/SSD3/maheen-data';
dir_meta='/home/SSD3/maheen-data/horse_project/ft_horse_allKp_gtWarp_halfBack_1e-3'

range_batches=range(1,2);
range_images=range(1,101);

img_caption_pre=['1680'];
img_dir=[os.path.join(dir_meta,img_caption_pre_curr,'test_images') for img_caption_pre_curr in img_caption_pre];

file_post=['.jpg','_org.jpg']
# ,'.jpg']
# ,'.jpg']
out_file_html=os.path.join(dir_meta,'1680')+'.html';
print out_file_html.replace(dir_server,'http://vision1.idav.ucdavis.edu:1000')
comparativeLossViz(img_dir,file_post,None,range_batches,range_images,out_file_html,dir_server,img_caption_pre);




http://vision1.idav.ucdavis.edu:1000/horse_project/ft_horse_allKp_gtWarp_halfBack_1e-3/1680.html


In [10]:
import util;

horse_data_path='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_train_allKP_minLoss.txt';
human_data_path='/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP_minLoss.txt';

out_file_horse='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_train_allKP_minLoss_dummy.txt';
out_file_human='/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP_minLoss_dummy.txt';

horse_data=util.readLinesFromFile(horse_data_path);
human_data=util.readLinesFromFile(human_data_path);
batch_size=64;
problem_batch=13-1;

start_idx=problem_batch*batch_size;
end_idx=(problem_batch+1)*batch_size
horse_batch_curr=horse_data[start_idx:end_idx];
human_batch_curr=human_data[start_idx:end_idx];
print len(horse_batch_curr),len(human_batch_curr);

util.writeFile(out_file_horse,horse_batch_curr)
util.writeFile(out_file_human,human_batch_curr);

64 64


In [40]:
import util;
import numpy as np;
import os;
import cv2;
import visualize;

def saveImWithAnno(im_path,npy_path,out_path):
    im=cv2.imread(im_path,1);
    # im=cv2.res tiize(im,(224,224));
    label=np.load(npy_path).astype(np.int);
    x=label[:,0];
    y=label[:,1];
    color=(0,0,255);
    colors=[(0,0,255),(0,255,0),(255,0,0),(255,255,0),(0,255,255)]

    for label_idx in range(len(x)):
        if label[label_idx,2]>0:
            cv2.circle(im,(x[label_idx],y[label_idx]),5,colors[label_idx],-1);
    cv2.imwrite(out_path,im);

dir_server='/home/SSD3/maheen-data';
click_str='http://vision1.idav.ucdavis.edu:1000'
out_file_horse='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_train_allKP_minLoss.txt';
out_file_human='/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP_minLoss.txt';

# out_file_horse='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss.txt';
# out_file_human='/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_val_allKP_minLoss.txt';

out_dir_meta=os.path.join(dir_server,'horse_project/temp/debug_gtWarp');

out_dir_test_images=os.path.join(out_dir_meta,'test_images');
out_file_plot=os.path.join(out_dir_test_images,'loss_plot.jpg');
loss_file=os.path.join(out_dir_test_images,'loss_final_val_ind.npy');

print out_dir_test_images;
horse_data=util.readLinesFromFile(out_file_horse);
human_data=util.readLinesFromFile(out_file_human)

loss=np.load(loss_file);



idx_sort=np.argsort(loss);
idx_sort=idx_sort[::-1];
idx_sort=idx_sort[:100];
losses_sorted=loss[idx_sort];
batch_size=100;


visualize.plotSimple([(range(losses_sorted.size),losses_sorted)],\
                     out_file_plot,title='Losses',xlabel='Index',ylabel='Loss');

ordered_pres=[];

for idx_idx,idx_prob in enumerate(idx_sort):
    horse_im,horse_npy=horse_data[idx_prob].split(' ');
    human_im,human_npy=human_data[idx_prob].split(' ');
    batch_num=idx_prob/batch_size;
    batch_num=batch_num+1;
    im_num=(idx_prob%batch_size)+1;
    str_im=str(batch_num)+'_'+str(im_num)
    out_file_pre=os.path.join(out_dir_test_images,str_im);
    saveImWithAnno(horse_im,horse_npy,out_file_pre+'_horse_gt.jpg');
    saveImWithAnno(human_im,human_npy,out_file_pre+'_human_gt.jpg');
    ordered_pres.append(out_file_pre);

    
out_file_html=os.path.join(out_dir_meta,'train_check.html');
out_file_record=os.path.join(out_dir_meta,'train_pairs.txt');
img_paths=[];
captions=[];
human_horse_pairs=[];
posts=['_horse_gt','_human_gt','','_org'];
for idx_curr,file_curr in enumerate(ordered_pres):
    img_row_curr=[util.getRelPath(file_curr+post_curr+'.jpg',dir_server) for post_curr in posts];
    caption_row_curr=[str(losses_sorted[idx_curr])[:7]+' '+post_curr for post_curr in posts]
    img_paths.append(img_row_curr);
    captions.append(caption_row_curr);
    
    human_horse_curr=[file_curr+post_curr+'.jpg' for post_curr in posts[:2]];
    human_horse_curr=' '.join(human_horse_curr);
    human_horse_pairs.append(human_horse_curr);
    
    if losses_sorted[idx_curr]<0.05:
        break;


visualize.writeHTML(out_file_html,img_paths,captions);
print out_file_html.replace(dir_server,click_str);

util.writeFile(out_file_record,human_horse_pairs);


/home/SSD3/maheen-data/horse_project/temp/debug_gtWarp/test_images
http://vision1.idav.ucdavis.edu:1000/horse_project/temp/debug_gtWarp/train_check.html


In [72]:
import util;
import numpy as np;
import os;
import cv2;
import visualize;

def saveImWithAnno(im_path,npy_path,out_path):
    im=cv2.imread(im_path,1);
    # im=cv2.res tiize(im,(224,224));
    label=np.load(npy_path).astype(np.int);
    x=label[:,0];
    y=label[:,1];
    color=(0,0,255);
    colors=[(0,0,255),(0,255,0),(255,0,0),(255,255,0),(0,255,255)]

    for label_idx in range(len(x)):
        if label[label_idx,2]>0:
            cv2.circle(im,(x[label_idx],y[label_idx]),5,colors[label_idx],-1);
    cv2.imwrite(out_path,im);

def getImNumAndCheck(dir_curr):
    txt_files=[file_curr for file_curr in os.listdir(dir_curr) if file_curr.endswith('.txt')];
    content=[int(util.readLinesFromFile(os.path.join(dir_curr,file_curr))[0]) for file_curr in txt_files];
    content=np.array(content);
    im_nums=[file_curr[:file_curr.rindex('.')] for file_curr in txt_files];
    im_nums=np.array(im_nums);
    return im_nums,content;
    
dir_server='/home/SSD3/maheen-data';
click_str='http://vision1.idav.ucdavis.edu:1000'
out_dir_meta=os.path.join(dir_server,'horse_project/temp/debug_gtWarp');

out_file_horse='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_train_allKP_minLoss.txt';
out_file_human='/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP_minLoss_noIm.txt';
out_dir_test_images=os.path.join(out_dir_meta,'test_images');
dir_check_txt=os.path.join(out_dir_meta,'train_pairs_val_no_val');

# out_file_horse='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss.txt';
# out_file_human='/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_val_allKP_minLoss_noIm.txt';
# out_dir_test_images=os.path.join(out_dir_meta,'test_images_val');
# dir_check_txt=os.path.join(out_dir_meta,'test_pairs_val_no_val');


out_file_horse_clean=out_file_horse[:out_file_horse.rindex('.')]+'_clean.txt';
out_file_human_clean=out_file_human[:out_file_human.rindex('.')]+'_clean.txt';

out_file_plot=os.path.join(out_dir_test_images,'loss_plot.jpg');
loss_file=os.path.join(out_dir_test_images,'loss_final_val_ind.npy');





# out_file_horse='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss.txt';
# out_file_human='/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_val_allKP_minLoss.txt';


im_nums,content = getImNumAndCheck(dir_check_txt);
im_nums = im_nums[content==2];

print out_dir_test_images;
horse_data=util.readLinesFromFile(out_file_horse);
human_data=util.readLinesFromFile(out_file_human)

loss=np.load(loss_file);

idx_sort=np.argsort(loss);
idx_sort=idx_sort[::-1];
idx_sort=idx_sort[:100];
losses_sorted=loss[idx_sort];
batch_size=100;

img_path_pairs=[];
captions=[];
problem_idx=[];
for im_num in im_nums:
    im_num_split=im_num.split('_');
    
    batch_num=int(im_num_split[0]);

    d=(batch_num-1)*batch_size;
    rem=int(im_num_split[1]);
    idx_num=d+rem;
    idx_num=idx_num-1;
    def_path=os.path.join(out_dir_test_images,im_num+'.jpg');
    n_def_path=horse_data[idx_num].split(' ')[0];
    imgs_curr=[def_path,n_def_path];
    img_path_pairs.append([util.getRelPath(file_curr,dir_server) for file_curr in imgs_curr]);
    captions.append([os.path.split(file_curr)[1]+' '+str(idx_num) for file_curr in imgs_curr])
    problem_idx.append(idx_num);

old_data=zip(horse_data,human_data);
new_data=[old_data[idx] for idx in range(len(horse_data)) if idx not in problem_idx];
horse_data_new,human_data_new=zip(*new_data);
print len(horse_data),len(horse_data_new);

out_file_html=os.path.join(out_dir_meta,'check.html');
visualize.writeHTML(out_file_html,img_path_pairs,captions);
print out_file_html.replace(dir_server,click_str);

util.writeFile(out_file_horse_clean,horse_data_new);
util.writeFile(out_file_human_clean,human_data_new);


/home/SSD3/maheen-data/horse_project/temp/debug_gtWarp/test_images
3546 3531
http://vision1.idav.ucdavis.edu:1000/horse_project/temp/debug_gtWarp/check.html


In [84]:
import util;
import os;
import numpy as np;

horse_dir='/home/SSD3/maheen-data/horse_project/data_check/horse';
human_dir='/home/SSD3/maheen-data/horse_project/data_check/aflw';

clean_name=['matches_5_train_allKP_minLoss_clean.txt','matches_5_train_allKP_minLoss_noIm_clean.txt'];
org_name=['matches_5_train_allKP.txt','matches_5_train_allKP_noIm.txt'];
new_name=[clean_name_curr[:clean_name_curr.rindex('.')]+'_full.txt' for clean_name_curr in clean_name];


org_data=util.readLinesFromFile(os.path.join(horse_dir,org_name[0]));
clean_data=util.readLinesFromFile(os.path.join(horse_dir,clean_name[0]));
clean_data=np.array(clean_data);
org_data=np.array(org_data);
index_new=np.in1d(org_data,clean_data);
print np.sum(index_new),len(clean_data),len(org_data);
index_new=np.where(index_new)[0]
print index_new.shape;

for dir_curr in [horse_dir,human_dir]:
    for idx_curr in range(len(org_name)):
        in_file=os.path.join(dir_curr,org_name[idx_curr])
        if not os.path.exists(in_file):
            continue;
        org_data=util.readLinesFromFile(in_file);
        new_data=[org_data[idx_select] for idx_select in index_new];
        print len(org_data),len(new_data);
        out_file=os.path.join(dir_curr,new_name[idx_curr]);
        util.writeFile(out_file,new_data);
        print in_file,out_file;

17494 3531 17537
(17494,)
17537 17494
/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_train_allKP.txt /home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_train_allKP_minLoss_clean_full.txt
17537 17494
/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP.txt /home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP_minLoss_clean_full.txt
17537 17494
/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP_noIm.txt /home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP_minLoss_noIm_clean_full.txt


In [87]:
img_dirs=['/home/SSD3/maheen-data/temp/checkRotation'];
file_post=['_horse.jpg','_human.jpg'];
loss_post=None;
range_batches=[1];
range_images=range(1,4);
out_file_html=os.path.join(img_dirs[0],'check.html');
dir_server='/home/SSD3/maheen-data';


comparativeLossViz(img_dirs,file_post,loss_post,range_batches,range_images,out_file_html,dir_server);
print out_file_html.replace(dir_server,'http://vision1.idav.ucdavis.edu:1000');

http://vision1.idav.ucdavis.edu:1000/temp/checkRotation/check.html
